# Investigate: Non-English Attachments on English pages
Notebook explores GOV.UK pages that are non-English attachments but are being marked as English, the default choice.

This is part of the Accessibility work to ensure compliance with WCAG. These attachments are currently WCAG fails because screen-reading software that the visually impaired use to read GOV.UK pages will suggest these attachments are English and thus the person will download it, when it the attachment is not actually in English. The consequence is that they will then have to download another attachment, so the page is less accessible.

In particular, this notebook lay the code basis so that it can go into the report runner.

## Postulation
The approach this notebook will take is to identify a column in the pre-processed content store that has the attachment in. We will define this by looking for text that ends in something such as `.pdf`. There are two directions that we can then take:
1. Detect language of attachment via the name
     + Is easiest method
     + Less reliable because names of attachments are typically short and language detection works less effectively when it has less language to scan.
1. Detect language of contents of attachment
     + Harder as you need to read the attachments in bulk
     + All sorts of different attachments such as `.pdf`, `.doc`, `.csv`, `.html` so need a variety of ways to read the contents
     + More accurate as will be working with extra text
     
Option (2.) is on the backburner for now because we would need the full page paths to the attachments and it would be really slow to download all the attachments and read their contents. Can construct it like something below:
```
str('govuk/' + base_path + `file_name`)
```

In [ ]:
import os
import time

import pandas as pd
import numpy as np
from multiprocessing import Pool

from langdetect import detect_langs

In [ ]:
def func_detectlangs(text):
    """Detects language of a text, moving onto next text if an error is thrown
    
    :param text: A string to detect the language of
    :return: A list returning the language detected and confidence score associated to it
    
    """

    try:
        return detect_langs(text)
    except:
        pass

In [ ]:
def func_detectlang_df(df):
    """Apply funcs_detectlangs() function on dataframe columns
    
    :param df: A dataframe with `text` column to apply func_detectlangs() on
    :return: A dataframe with extra column `text_lang` that identifies what language and the level of confidence of the text passed in
    
    """
    df['text_lang'] = df['text'].apply(lambda text: func_detectlangs(text))
    return df

def func_detectlang_pool(df, func, n_cores):
    """Parallelises the func_detectlang_df function
    
    :param df: A dataframe to pass into `func`
    :param func: A function to apply to dataframe
    :param n_cores: Number of cores to parallelise on
    """
    
    df_split = np.array_split(df, n_cores)

    p = Pool(processes = n_cores)
    
    df = pd.concat(p.map(func, df_split))
    
    p.close()
    p.join()
    
    return df

In [ ]:
# number of cores on machine
n_cores = os.cpu_count() - 1

## Load Data
Data used in this will be all the content on GOV.UK that exist on 6th August 2020.

Due to the sheer size of the data, need to pre-specify column headings and which columns are dates to make the import process:
- Work
- Work relatively quickly

In [ ]:
# create dictionaries and headers to specify dtype and date columns
dict_header = {'base_path':object,
               'content_id':object,
               'title':object,
               'description':object,
               'publishing_app':object,
               'document_type':object,
               'details':object,  
               'text':object,
               'organisations':object,  
               'taxons':object,
               'step_by_steps':object,
               'details_parts':object,  
               'first_published_at':object,
               'public_updated_at':object,
               'updated_at':object,
               'finder':object,
               'facet_values':object,  
               'facet_groups':object,
               'has_brexit_no_deal_notice':bool,
               'withdrawn':bool,
               'withdrawn_at':object,
               'withdrawn_explanation':object}
list_header_date = ['first_published_at',
                    'public_updated_at',
                    'updated_at',
                    'withdrawn_at']

# load data
df = pd.read_csv(filepath_or_buffer='../data/preprocessed_content_store_200820.csv.gz',
                 compression='gzip',
                 encoding='utf-8',
                 sep='\t',
                 header=0,
                 names=list(dict_header.keys()),
                 dtype=dict_header,
                 parse_dates=list_header_date)

In [ ]:
del dict_header, list_header_date

To find webpages with attachments on, we assume the following (based on a few case examples):
1. They have a non-empty list in the `'documents': [...]` element
1. They have a non-empty list in the `'attachments': [...]` element

Not perfect though, still have pages that don't have any attachments in them.

In [ ]:
# have attachments in `details` column, under 'attachments'
df['details_document_exists'] = df['details'].str.contains('\'documents\'\: \[', na = False)
df['details_attachment_exists'] = df['details'].str.contains('\'attachments\'\: \[', na = False)
df_attachment = df.query('details_document_exists == True & details_attachment_exists == True')

In [ ]:
df_attachment[['title', 'publishing_app', 'document_type', 'details', 'text', 'public_updated_at']].sample(n = 5, random_state = 42)

***

## Preprocessing
Let's extract the file names from the urls so that we can start detecting the language. Will do this in two main stages:
1. Extract the urls from the HTML code
1. Extract the file names and extensions from the urls

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
# test case
#/government/publications/measles-mumps-and-rubella-lab-confirmed-cases-in-england-2019
text_test = df_attachment[df_attachment['base_path'].str.contains("/government/publications/dart-charge-bulletin-3-advice-for-foreign-hgv-drivers")].iloc[0]['details']

In [ ]:
soup = BeautifulSoup(text_test, 'html.parser')
print(soup.prettify())

In [ ]:
for text in soup.find_all('div', class_ = 'attachment-details'):
    for title in text.find_all('h2', class_ = 'title'):
        print(title.get_text())

In [ ]:
def extract_attachment_titles(html):
    """Extracts all the attachment titles from GOV.UK pages
    
    :param html: String of the HTML code for the GOV.UK page being passed in
    :return: list of all the attachment titles that were extracted from GOV.UK page
    
    """
    
    # pass html into BeautifulSoup class to apply methods on it
    soup = BeautifulSoup(html, 'html.parser')
    
    # initialise list to store results
    list_title = []
    
    # extract all text from `h2` element with class description `title` 
    # nested in `div` element with class description `attachment-details`
    for text in soup.find_all('div', class_ = 'attachment-details'):
        for title in text.find_all('h2', class_ = 'title'):
            list_title.append(title.get_text())

    
    return list_title

In [ ]:
# check if worked correctly
df_extract = df_attachment[['base_path', 'text', 'details']].copy()
df_extract['attachment_title'] = df_extract['details'].apply(extract_attachment_titles)
df_extract.sample(n = 1000, random_state = 42).to_csv('../data/sample_attachments.csv')